In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import numpy as np
import os
from keras import backend as K
from keras.layers import Conv2D, Dropout, LSTM, BatchNormalization, Input,Activation, MaxPool2D, Flatten, Dense,TimeDistributed
from keras.models import Model, load_model
from keras import metrics 

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
VIDEOS_DIR = './Videos/'
IMAGES_DIR = './Images/'

In [3]:
classes = list(os.listdir(VIDEOS_DIR))
print(classes)

['Kicking', 'Riding-Horse', 'Running', 'SkateBoarding', 'Swing-Bench', 'Lifting', 'Swing-Side', 'Walking', 'Golf-Swing']


In [4]:
class_to_index = {}
for i in range(len(classes)):
    class_to_index[classes[i]] = i
class_to_index

{'Golf-Swing': 8,
 'Kicking': 0,
 'Lifting': 5,
 'Riding-Horse': 1,
 'Running': 2,
 'SkateBoarding': 3,
 'Swing-Bench': 4,
 'Swing-Side': 6,
 'Walking': 7}

In [5]:
videos = []
for x in classes:
    videos.append(list(os.listdir(VIDEOS_DIR+x+'/')))
print(videos)

[['006', '017', '003', '016', '009', '013', '005', '012', '010', '015', '014', '007', '004', '011', '002', '001'], ['006', '003', '009', '005', '010', '007', '004', '002', '008', '001'], ['006', '009', '005', '010', '007', '004', '002', '008', '001'], ['006', '003', '009', '005', '010', '007', '004', '002', '008', '001'], ['006', '017', '003', '016', '009', '013', '005', '012', '010', '015', '014', '007', '004', '011', '002', '008', '001'], ['003', '005', '004', '002', '001'], ['006', '003', '009', '005', '010', '007', '004', '011', '002', '008', '001'], ['006', '017', '003', '016', '009', '013', '005', '012', '010', '015', '014', '007', '019', '004', '011', '018', '002', '008', '001'], ['006', '003', '009', '013', '005', '012', '010', '014', '007', '004', '011', '002', '008', '001']]


### Function Block

In [6]:
def permute(X,Y):
    train_size = X.shape[0]
    permutation_train = np.random.permutation(train_size)
    X = X[permutation_train]
    Y = Y[permutation_train]
    return X,Y

def load_image(path,image_size):
    image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, image_size)
    return image

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

def model_predict(model,images):
    output = K.function([model.layers[0].input,K.learning_phase()],
                        [model.layers[13].output])
    return output([images,0])[0]

def pad(X_cnn,max_len):
    features_len = X_cnn.shape[1]
    length = X_cnn.shape[0]
    X_cnn = list(X_cnn)
    pad_arr = [0 for i in range(features_len)]
    for i in range(max_len-length):
        X_cnn.append(pad_arr)
    return np.array(X_cnn) 

### Build Dataset

In [7]:
def build_dataset_for_lstm_strided(image_size, stride = 10, max_len = 40):
    
    model = load_model('models/Conv/17epochs_valacc_94.h5')
    
    X_train_images = []
    Y_train_images = []
    
    for i in range(len(classes)):
        cls = classes[i]
        for j in range(len(videos[i])):
            vid = videos[i][j]
            video_r = VIDEOS_DIR+cls+'/'+ vid +'/'
            image_r = IMAGES_DIR+cls+'/'+ vid +'/'
            
            filelist = sorted(list(os.listdir(image_r)))
            X_train_images_class = []
            
            for file in filelist:
                if file.endswith(".png"):
                    image = load_image(image_r+file,image_size)
                    X_train_images_class.append(image)
            X_cnn = model_predict(model,np.array(X_train_images_class))
            print(X_cnn.shape)
            
            for k in range(0,X_cnn.shape[0],stride):
                lower = k
                upper = min(X_cnn.shape[0],k+max_len)
                if upper == X_cnn.shape[0]:
                    X_train_images.append(pad(X_cnn[lower:upper],max_len))
                    Y_train_images.append(i)
                    print("Padded frames" , lower , "to" , upper)
                    break
                else:
                    X_train_images.append(X_cnn[lower:upper])
                    Y_train_images.append(i)
                    print("Added frames" , lower , "to" , upper)

           
            print("Processed",videos[i][j],"of","class",classes[i])
    return X_train_images,Y_train_images

### Conv Model

In [8]:
def build_model(input_shape):
    X_input = Input(input_shape, name = "Input")
    
    X = BatchNormalization(name = 'BatchNorm_1')(X_input)
    X = Conv2D(32, (7, 7), strides = (5, 5), name="Conv_1a", padding="same")(X)
    X = Activation('relu')(X)
    
    X = Conv2D(32, (3, 3), name = "Conv_1b", padding="same")(X)
    X = Activation('relu')(X)
    X = MaxPool2D((2, 2), name = "Pool_1")(X)
    X = Dropout(0.2)(X)
    
    X = Conv2D(32, (3, 3), name ="Conv_2", padding = "same")(X)
    X = Activation('relu')(X)
    X = MaxPool2D((4, 4), name = "Pool_2")(X)
    X = Dropout(0.2)(X)
    
    X = Conv2D(8,(1,1), name='Conv_1x1')(X)
    
    X = Flatten()(X)
    X = Dropout(0.2)(X)
    
    return Model(X_input, X)

In [9]:
def CNN_model(prev_model):
    X = Dense(9,activation='softmax',name='final')(prev_model.output)
    return Model(prev_model.input, X)

In [10]:
model = build_model((172, 172, 3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 172, 172, 3)       0         
_________________________________________________________________
BatchNorm_1 (BatchNormalizat (None, 172, 172, 3)       12        
_________________________________________________________________
Conv_1a (Conv2D)             (None, 35, 35, 32)        4736      
_________________________________________________________________
activation_1 (Activation)    (None, 35, 35, 32)        0         
_________________________________________________________________
Conv_1b (Conv2D)             (None, 35, 35, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 35, 35, 32)        0         
_________________________________________________________________
Pool_1 (MaxPooling2D)        (None, 17, 17, 32)        0         
__________

In [11]:
cnn = CNN_model(model)
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 172, 172, 3)       0         
_________________________________________________________________
BatchNorm_1 (BatchNormalizat (None, 172, 172, 3)       12        
_________________________________________________________________
Conv_1a (Conv2D)             (None, 35, 35, 32)        4736      
_________________________________________________________________
activation_1 (Activation)    (None, 35, 35, 32)        0         
_________________________________________________________________
Conv_1b (Conv2D)             (None, 35, 35, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 35, 35, 32)        0         
_________________________________________________________________
Pool_1 (MaxPooling2D)        (None, 17, 17, 32)        0         
__________

In [12]:
cnn.compile(loss='categorical_crossentropy', 
            metrics=['accuracy'], 
            optimizer='adam')

### RNN Model

In [13]:
def rnn_model(input_shape):
    X_input = Input(input_shape)
    X = LSTM(32, return_sequences=True)(X_input)
    X = LSTM(32, return_sequences=False)(X)
    X = Dense(9,activation='softmax')(X)
    return Model(X_input, X)

In [14]:
rnn = rnn_model((40,128))
rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 32)            20608     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 297       
Total params: 29,225
Trainable params: 29,225
Non-trainable params: 0
_________________________________________________________________


In [15]:
rnn.compile(loss='categorical_crossentropy', 
            metrics=['accuracy'], 
            optimizer='adam')

In [16]:
X = None
Y = None
try:
    X = np.load('Numpy/LSTM_Strided/train_X.npy')
    Y = np.load('Numpy/LSTM_Strided/train_Y.npy')
except FileNotFoundError:
    X,Y = build_dataset_for_lstm_strided((172,172))
    X = np.array(X)
    Y = convert_to_one_hot(np.array(Y),9)

print("Shape X",X.shape)
print("Shape Y",Y.shape)
X_train_rnn,Y_train_rnn = permute(X,Y)

Shape X (419, 40, 128)
Shape Y (419, 9)


In [17]:
rnn.fit(X_train_rnn, Y_train_rnn, epochs=100, batch_size = X_train_rnn.shape[0], validation_split=0.2)

Train on 335 samples, validate on 84 samples
Epoch 1/500
335/335 [==============================] - 1s 2ms/step - loss: 2.1607 - acc: 0.0985 - val_loss: 2.1107 - val_acc: 0.1548
Epoch 2/500
335/335 [==============================] - 0s 264us/step - loss: 2.0946 - acc: 0.1433 - val_loss: 2.0608 - val_acc: 0.3214
Epoch 3/500
335/335 [==============================] - 0s 264us/step - loss: 2.0361 - acc: 0.2746 - val_loss: 2.0158 - val_acc: 0.3810
Epoch 4/500
335/335 [==============================] - 0s 269us/step - loss: 1.9801 - acc: 0.3881 - val_loss: 1.9707 - val_acc: 0.4048
Epoch 5/500
335/335 [==============================] - 0s 270us/step - loss: 1.9271 - acc: 0.4388 - val_loss: 1.9295 - val_acc: 0.4643
Epoch 6/500
335/335 [==============================] - 0s 265us/step - loss: 1.8802 - acc: 0.5104 - val_loss: 1.8901 - val_acc: 0.5238
Epoch 7/500
335/335 [==============================] - 0s 281us/step - loss: 1.8352 - acc: 0.5642 - val_loss: 1.8521 - val_acc: 0.5952
Epoch 8/500


Epoch 61/500
335/335 [==============================] - 0s 289us/step - loss: 0.2594 - acc: 0.9672 - val_loss: 0.3396 - val_acc: 0.9405
Epoch 62/500
335/335 [==============================] - 0s 255us/step - loss: 0.2506 - acc: 0.9672 - val_loss: 0.3334 - val_acc: 0.9405
Epoch 63/500
335/335 [==============================] - 0s 272us/step - loss: 0.2422 - acc: 0.9672 - val_loss: 0.3280 - val_acc: 0.9405
Epoch 64/500
335/335 [==============================] - 0s 266us/step - loss: 0.2342 - acc: 0.9672 - val_loss: 0.3230 - val_acc: 0.9405
Epoch 65/500
335/335 [==============================] - 0s 298us/step - loss: 0.2268 - acc: 0.9672 - val_loss: 0.3179 - val_acc: 0.9405
Epoch 66/500
335/335 [==============================] - 0s 308us/step - loss: 0.2198 - acc: 0.9672 - val_loss: 0.3126 - val_acc: 0.9405
Epoch 67/500
335/335 [==============================] - 0s 303us/step - loss: 0.2128 - acc: 0.9672 - val_loss: 0.3072 - val_acc: 0.9405
Epoch 68/500
335/335 [==========================

335/335 [==============================] - 0s 315us/step - loss: 0.0409 - acc: 1.0000 - val_loss: 0.2330 - val_acc: 0.9524
Epoch 122/500
335/335 [==============================] - 0s 292us/step - loss: 0.0395 - acc: 1.0000 - val_loss: 0.2330 - val_acc: 0.9524
Epoch 123/500
335/335 [==============================] - 0s 319us/step - loss: 0.0382 - acc: 1.0000 - val_loss: 0.2330 - val_acc: 0.9524
Epoch 124/500
335/335 [==============================] - 0s 297us/step - loss: 0.0370 - acc: 1.0000 - val_loss: 0.2328 - val_acc: 0.9524
Epoch 125/500
335/335 [==============================] - 0s 267us/step - loss: 0.0358 - acc: 1.0000 - val_loss: 0.2325 - val_acc: 0.9524
Epoch 126/500
335/335 [==============================] - 0s 265us/step - loss: 0.0346 - acc: 1.0000 - val_loss: 0.2319 - val_acc: 0.9524
Epoch 127/500
335/335 [==============================] - 0s 312us/step - loss: 0.0336 - acc: 1.0000 - val_loss: 0.2312 - val_acc: 0.9524
Epoch 128/500
335/335 [==============================] 

Epoch 181/500
335/335 [==============================] - 0s 366us/step - loss: 0.0122 - acc: 1.0000 - val_loss: 0.3090 - val_acc: 0.9405
Epoch 182/500
335/335 [==============================] - 0s 337us/step - loss: 0.0121 - acc: 1.0000 - val_loss: 0.3092 - val_acc: 0.9405
Epoch 183/500
335/335 [==============================] - 0s 290us/step - loss: 0.0120 - acc: 1.0000 - val_loss: 0.3091 - val_acc: 0.9405
Epoch 184/500
335/335 [==============================] - 0s 328us/step - loss: 0.0119 - acc: 1.0000 - val_loss: 0.3088 - val_acc: 0.9405
Epoch 185/500
335/335 [==============================] - 0s 339us/step - loss: 0.0117 - acc: 1.0000 - val_loss: 0.3080 - val_acc: 0.9405
Epoch 186/500
335/335 [==============================] - 0s 308us/step - loss: 0.0116 - acc: 1.0000 - val_loss: 0.3067 - val_acc: 0.9405
Epoch 187/500
335/335 [==============================] - 0s 302us/step - loss: 0.0115 - acc: 1.0000 - val_loss: 0.3047 - val_acc: 0.9405
Epoch 188/500
335/335 [==================

Epoch 241/500
335/335 [==============================] - 0s 294us/step - loss: 0.0074 - acc: 1.0000 - val_loss: 0.2922 - val_acc: 0.9405
Epoch 242/500
335/335 [==============================] - 0s 296us/step - loss: 0.0074 - acc: 1.0000 - val_loss: 0.2905 - val_acc: 0.9405
Epoch 243/500
335/335 [==============================] - 0s 333us/step - loss: 0.0073 - acc: 1.0000 - val_loss: 0.2888 - val_acc: 0.9405
Epoch 244/500
335/335 [==============================] - 0s 295us/step - loss: 0.0073 - acc: 1.0000 - val_loss: 0.2877 - val_acc: 0.9405
Epoch 245/500
335/335 [==============================] - 0s 305us/step - loss: 0.0072 - acc: 1.0000 - val_loss: 0.2867 - val_acc: 0.9405
Epoch 246/500
335/335 [==============================] - 0s 299us/step - loss: 0.0072 - acc: 1.0000 - val_loss: 0.2856 - val_acc: 0.9405
Epoch 247/500
335/335 [==============================] - 0s 286us/step - loss: 0.0071 - acc: 1.0000 - val_loss: 0.2847 - val_acc: 0.9405
Epoch 248/500
335/335 [==================

Epoch 301/500
335/335 [==============================] - 0s 429us/step - loss: 0.0052 - acc: 1.0000 - val_loss: 0.2602 - val_acc: 0.9405
Epoch 302/500
335/335 [==============================] - 0s 434us/step - loss: 0.0052 - acc: 1.0000 - val_loss: 0.2603 - val_acc: 0.9405
Epoch 303/500
335/335 [==============================] - 0s 440us/step - loss: 0.0052 - acc: 1.0000 - val_loss: 0.2604 - val_acc: 0.9405
Epoch 304/500
335/335 [==============================] - 0s 433us/step - loss: 0.0051 - acc: 1.0000 - val_loss: 0.2604 - val_acc: 0.9405
Epoch 305/500
335/335 [==============================] - 0s 448us/step - loss: 0.0051 - acc: 1.0000 - val_loss: 0.2605 - val_acc: 0.9405
Epoch 306/500
335/335 [==============================] - 0s 424us/step - loss: 0.0051 - acc: 1.0000 - val_loss: 0.2605 - val_acc: 0.9405
Epoch 307/500
335/335 [==============================] - 0s 439us/step - loss: 0.0050 - acc: 1.0000 - val_loss: 0.2605 - val_acc: 0.9405
Epoch 308/500
335/335 [==================

KeyboardInterrupt: 

In [ ]:
#rnn.save('models/LSTM_Strided/500ep_valacc_97_single_LSTM.h5')